In [102]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score,precision_score

import warnings
warnings.filterwarnings("ignore")

In [91]:
df = pd.read_excel('Data_08_02_2024.xlsx')

In [92]:
df.head()

,business_id,product_a,product_b,hard_closure,soft_closure,month_1,month_2,month_3,month_4,month_5,...,call_last_3_months,call_last_6_months,call_reason_account_setup_changes,call_reason_online_equipment,call_reason_other,call_reason_standalone_equipment,call_reason_transaction_batch_funding,Industry,Yrs_with_Chase,chargeback_txn_cnt
0,1000089098,1,0,0,1,0.00,4790.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Transportation, Communication, Utilities",NaN,1.0
1,1000089099,1,0,0,0,158.00,2263.00,5536.00,115.00,163.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business Services,35.3,NaN
2,1000089100,1,0,0,1,0.00,1165.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Construction,31.7,NaN
3,1000089101,1,0,0,0,1083.15,1052.97,729.71,426.27,992.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business Services,24.0,NaN
4,1000089102,1,0,1,0,0.00,0.00,0.00,3611.54,3390.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Construction,60.5,NaN


In [93]:
df.isna().sum()

business_id                                  0
product_a                                    0
product_b                                    0
hard_closure                                 0
soft_closure                                 0
                                         ...  
call_reason_standalone_equipment         65779
call_reason_transaction_batch_funding    65735
Industry                                    28
Yrs_with_Chase                           17846
chargeback_txn_cnt                       57239
Length: 186, dtype: int64

In [94]:
df_copy = df.copy()

In [95]:
df.shape

(66077, 186)

In [96]:
threshold = len(df) * 0.20 # Calculate 20% of the total number of rows
dropped_columns = list(df.columns[df.isna().sum() > threshold])
df = df.dropna(thresh=threshold, axis=1)
print("Columns deleted:", dropped_columns)

Columns deleted: ['comp_month_1', 'comp_month_2', 'comp_month_3', 'comp_month_4', 'comp_month_5', 'comp_month_6', 'comp_month_7', 'comp_month_8', 'comp_month_9', 'comp_month_10', 'comp_month_11', 'comp_month_12', 'comp_month_13', 'comp_month_14', 'comp_month_15', 'comp_month_16', 'comp_month_17', 'comp_month_18', 'comp_month_19', 'comp_month_20', 'comp_month_21', 'comp_month_22', 'comp_month_23', 'comp_month_24', 'comp_month_25', 'comp_month_26', 'comp_month_27', 'comp_month_28', 'comp_month_29', 'comp_month_30', 'comp_month_31', 'comp_month_32', 'comp_month_33', 'comp_month_34', 'comp_month_35', 'comp_month_36', 'comp_month_37', 'comp_tpv_last_30_days', 'comp_tpv_last_60_days', 'comp_tpv_last_90_days', 'comp_tpv_last_120_days', 'comp_avg_monthly_tpv', 'comp_active_months', 'sales_volume', 'employees_total', 'Business_Age_Yrs', 'economic_area_of_influence', 'status_indicator', 'manufacturing_indicator', 'publicprivate_indicator', 'legal_status', 'hierarchy_code', 'number_of_family_memb

In [97]:
df.shape

(66077, 148)

In [98]:
# status_indicator,economic_area_of_influence,legal_status,hierarchy_code

In [99]:
df.drop(columns='business_id',inplace=True)

In [100]:
y = df[['hard_closure' ,'soft_closure']]
X = df.drop(columns=['hard_closure' ,'soft_closure'])

In [101]:
y

,hard_closure,soft_closure
0,0,1
1,0,0
2,0,1
3,0,0
4,1,0
...,...,...
66072,0,0
66073,0,1
66074,0,1
66075,0,0


In [103]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import pandas as pd

# Define column names based on their types
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Define transformers for numeric and categorical features
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply the preprocessing pipeline to the DataFrame
processed_data = preprocessor.fit_transform(X)

# Convert the processed data back to a DataFrame (optional)
X = pd.DataFrame(processed_data)


In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [106]:
y['hard_closure'].value_counts()

0    59187
1     6890
Name: hard_closure, dtype: int64

In [107]:
y['soft_closure'].value_counts()

1    40921
0    25156
Name: soft_closure, dtype: int64

In [108]:
y_train

,hard_closure,soft_closure
18716,0,1
56011,0,0
6400,0,0
27951,1,1
43851,0,0
...,...,...
37194,0,1
6265,0,1
54886,0,1
860,0,1


In [109]:
# forest = RandomForestClassifier(random_state=1)
# multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
# multi_target_forest.fit(X_train, y_train)

In [110]:
# def get_performance(y_pred):
    
#     print('Hard Closure')
#     print(classification_report(y_test['hard_closure'],y_pred[:,0]))
    
#     print('Soft Closure')
#     print(classification_report(y_test['soft_closure'],y_pred[:,1]))

In [112]:
from sklearn.metrics import classification_report,accuracy_score
from sklearn import ensemble,neighbors,tree

from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# https://scikit-learn.org/stable/modules/multiclass.html
MLA = [
    ensemble.RandomForestClassifier(random_state = 1,n_jobs=-1),
    ensemble.ExtraTreesClassifier(n_jobs=-1),
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier()
]

MLA_columns = ['MLA_names', 'Hard_Test_Accuracy','Hard_Test_F1','Hard_Test_Precision',
                            'Soft_Test_Accuracy','Soft_Test_F1','Soft_Test_Precision']

def apply_models(X_train,y_train,X_test,y_test):
    MLA_compare = pd.DataFrame(columns = MLA_columns)

#     MLA_Predict = y_test.copy()
    row_index = 0
    for alg in tqdm(MLA):
        try:
            MLA_name = alg.__class__.__name__
            MLA_compare.loc[row_index, 'MLA_names'] = MLA_name
#             MLA_compare.loc[row_index, 'MLA_parameters'] = str(alg.get_params())
            # cv_results = model_selection.cross_validate(alg, X_train, y_train, cv=3, return_train_score = True)
            # MLA_compare.loc[row_index, 'MLA_Time'] = cv_results['fit_time'].mean()

            # MLA_compare.loc[row_index, 'MLA_Test_Accuracy_3*STD'] = cv_results['test_score'].std()*3

            alg.fit(X_train, y_train)
            y_pred = alg.predict(X_test)
           
            ############## Hard closure results ###########
            acc = accuracy_score(y_test['hard_closure'],y_pred[:,0])
            f1 = f1_score(y_test['hard_closure'],y_pred[:,0])
            precision = precision_score(y_test['hard_closure'],y_pred[:,0])
            
            MLA_compare.loc[row_index, 'Hard_Test_Accuracy'] = acc
            MLA_compare.loc[row_index, 'Hard_Test_F1'] = f1
            MLA_compare.loc[row_index, 'Hard_Test_Precision'] = precision
            
            ########## Soft closure results ############
            acc = accuracy_score(y_test['hard_closure'],y_pred[:,1])
            f1 = f1_score(y_test['hard_closure'],y_pred[:,1])
            precision = precision_score(y_test['hard_closure'],y_pred[:,1])
            
            MLA_compare.loc[row_index, 'Soft_Test_Accuracy'] = acc
            MLA_compare.loc[row_index, 'Soft_Test_F1'] = f1
            MLA_compare.loc[row_index, 'Soft_Test_Precision'] = precision
            
            row_index += 1
        except:
            print(type(alg).__name__)
#     MLA_compare.sort_values(by = 'MLA_Test_Accuracy', ascending = False, inplace = True)
    return MLA_compare

In [113]:
apply_models(X_train,y_train,X_test,y_test)

  0%|          | 0/4 [00:00<?, ?it/s]

,MLA_names,Hard_Test_Accuracy,Hard_Test_F1,Hard_Test_Precision,Soft_Test_Accuracy,Soft_Test_F1,Soft_Test_Precision
0,RandomForestClassifier,0.903148,0.222357,0.601974,0.459367,0.24908,0.14499
1,ExtraTreesClassifier,0.902164,0.231729,0.571848,0.455735,0.24893,0.144748
2,DecisionTreeClassifier,0.858127,0.332503,0.318337,0.459443,0.249107,0.145007
3,ExtraTreeClassifier,0.8523,0.293266,0.285211,0.458232,0.246792,0.14368


In [ ]:
# - Todo
# - Apply SMOTE to balance data, up sampling/ Down sampling
# - function to get prediction for had/soft

In [114]:
a = df[(df['soft_closure']==0)].sample(frac=1)
b = df[(df['soft_closure']==1)].sample(len(a))

merged_df = pd.concat([a,b])

In [115]:
merged_df

,product_a,product_b,hard_closure,soft_closure,month_1,month_2,month_3,month_4,month_5,month_6,...,90plus_days_delinquent_bc_past_6_months,90plus_days_delinquent_bs_past_6_months,30plus_days_delinquent_bc_past_12_months,30plus_days_delinquent_bs_past_12_months,60plus_days_delinquent_bc_past_12_months,60plus_days_delinquent_bs_past_12_months,90plus_days_delinquent_bc_past_12_months,90plus_days_delinquent_bs_past_12_months,Industry,Yrs_with_Chase
11330,1,0,0,0,7315.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Construction,3.0
63536,1,0,0,0,0.00,0.00,0.00,0.00,0.00,285.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Personal Services,0.9
56197,1,0,0,0,0.00,1.00,190.00,0.00,0.00,969.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Business Services,1.3
12538,1,0,0,0,0.00,0.00,0.00,0.00,130.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Transportation, Communication, Utilities",11.1
3205,1,0,0,0,8750.00,0.00,1076.25,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Construction,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8529,1,0,0,1,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Retail,3.2
23086,1,0,1,1,605.69,5606.91,3538.16,4074.31,2655.97,2878.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business Services,NaN
1509,1,0,0,1,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Construction,1.8
9936,1,0,0,1,0.00,7402.50,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business Services,16.7


In [116]:
y1 = merged_df[['hard_closure' ,'soft_closure']]
X1 = merged_df.drop(columns=['hard_closure' ,'soft_closure'])

In [117]:
# Apply the preprocessing pipeline to the DataFrame
processed_data = preprocessor.fit_transform(X1)

# Convert the processed data back to a DataFrame (optional)
X1 = pd.DataFrame(processed_data)

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.20, random_state=42)

In [119]:
apply_models(X_train,y_train,X_test,y_test)

  0%|          | 0/4 [00:00<?, ?it/s]

,MLA_names,Hard_Test_Accuracy,Hard_Test_F1,Hard_Test_Precision,Soft_Test_Accuracy,Soft_Test_F1,Soft_Test_Precision
0,RandomForestClassifier,0.910166,0.184116,0.658065,0.561662,0.265201,0.157624
1,ExtraTreesClassifier,0.908079,0.153705,0.6,0.564643,0.266041,0.158293
2,DecisionTreeClassifier,0.873795,0.345361,0.339412,0.561662,0.265201,0.157624
3,ExtraTreeClassifier,0.86346,0.254881,0.263749,0.551923,0.254341,0.150962
